# 환경 설정 & KoBERT 설치
- https://github.com/ChangZero/koBERT-finetuning-demo/blob/main/kobert_colab.ipynb

In [ ]:
# # install and restart
# !pip install mxnet
# !pip install gluonnlp==0.8.0
# !pip install tqdm pandas
# !pip install sentencepiece
# !pip install transformers
# !pip install torch>=1.8.1

# !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [ ]:
!pip install transformers
# !pip install torchmetrics

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.notebook import tqdm

# git clone data

In [ ]:
!git clone https://github.com/huijunam/CJ_AI_hackathon.git

Cloning into 'CJ_AI_hackathon'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 35 (delta 8), reused 13 (delta 3), pack-reused 0
Receiving objects: 100% (35/35), 3.85 MiB | 13.26 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [ ]:
import pandas as pd

chatbot_data_1 = pd.read_excel('/content/CJ_AI_hackathon/data/8f3202096f1387c5.xlsx')
chatbot_data_2 = pd.read_excel('/content/CJ_AI_hackathon/data/ad8cb80ea8623bbe.xlsx')

In [ ]:
chatbot_data_1.sample(n =10)

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
36070,미쳐돌아가고잇네 ㅇㅇㅇ,혐오,NaN,NaN,NaN,NaN,NaN
38311,사생아 떡검이 분수를 모르고 선무당 짓한것의 결과입니다.,혐오,NaN,NaN,NaN,NaN,NaN
35324,하~~~진심 박ㄹ혜 싫다,혐오,NaN,NaN,NaN,NaN,NaN
11108,이게 20 30대 남자 댓글임?,놀람,NaN,NaN,NaN,NaN,NaN
15261,내 개인정보 짱깨한테 다털리라고 저딴 쓰레기폰을 사겠냐ㅋㅋㅋ,분노,NaN,NaN,NaN,NaN,NaN
3112,고민입니다 ...,공포,NaN,NaN,NaN,NaN,NaN
5517,아이고오 성님 원래 싸움체질은 아닌갑소..,놀람,NaN,NaN,NaN,NaN,NaN
22051,보스턴 가는데 필리핀가기까지가 1년도더남아서머하고지내야할지몰르겟네여...,슬픔,NaN,NaN,NaN,NaN,NaN
11223,재산에 딱 절반만 줘라~,놀람,NaN,NaN,NaN,NaN,NaN
24198,나 저 두명 거기 어케 생겼는지 앎 ㅎㅎㅎ 님!! 포매니... 악!!!!!!,중립,NaN,NaN,NaN,NaN,NaN


In [ ]:
chatbot_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38594 entries, 0 to 38593
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Sentence    38594 non-null  object 
 1   Emotion     38594 non-null  object 
 2   Unnamed: 2  0 non-null      float64
 3   Unnamed: 3  0 non-null      float64
 4   Unnamed: 4  0 non-null      float64
 5   공포          7 non-null      object 
 6   5468        7 non-null      float64
dtypes: float64(4), object(3)
memory usage: 2.1+ MB


In [ ]:
chatbot_data_1.describe()

,Unnamed: 2,Unnamed: 3,Unnamed: 4,5468
count,0.0,0.0,0.0,7.000000
mean,NaN,NaN,NaN,10245.714286
std,NaN,NaN,NaN,12506.947639
min,NaN,NaN,NaN,4830.000000
25%,NaN,NaN,NaN,5348.000000
50%,NaN,NaN,NaN,5665.000000
75%,NaN,NaN,NaN,5967.500000
max,NaN,NaN,NaN,38594.000000


In [ ]:
chatbot_data_1.count()

Sentence      38594
Emotion       38594
Unnamed: 2        0
Unnamed: 3        0
Unnamed: 4        0
공포                7
5468              7
dtype: int64

In [ ]:
# 필요없는 열 삭제
chatbot_data_1= chatbot_data_1.iloc[:,:2]
chatbot_data_1

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포
...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오
38590,재미가 없으니 망하지,혐오
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오


In [ ]:
chatbot_data_1.isnull().sum() # 결측치 확인

Sentence    0
Emotion     0
dtype: int64

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

chatbot_data_1['Emotion'].value_counts()

행복    6037
놀람    5898
분노    5665
공포    5468
혐오    5429
슬픔    5267
중립    4830
Name: Emotion, dtype: int64

In [ ]:
chatbot_data_2.sample(n =10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,행복,중립,슬픔,공포,혐오,분노,놀람
13384,NaN,만나서 얘기하자. 너한테 꼭 보여줄게 있어.,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13713,NaN,잠깐씩 이래. 정신 좀 돌아오면 또 괜찮아. 멀쩡해. 그래두 우리 집에서 속 젤 편...,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7492,NaN,둘째. 영어 선생님의 무단 광고 사건은 아시죠?,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9957,S,돌 던졌수.,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20629,S,이게 뭔데?,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47640,NaN,끝내주게 돌아가고 있어요.,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26526,S,쓰레기통 뒹구는 게 취미냐?,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43811,NaN,"그렇게 못믿으면서, 어떻게 2년씩이나 해외에 나가 있겠다고 했나 친구!",중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31565,NaN,상황은 어렵게 됐는데...다 밝혀지니까...뭐!! 맘은 편하네요.,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22219,NaN,이쯤되니 아주 가슴이 미어지네요.,슬픔,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
chatbot_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55629 entries, 0 to 55628
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  10001 non-null  object 
 1   Unnamed: 1  55628 non-null  object 
 2   Unnamed: 2  55622 non-null  object 
 3   Unnamed: 3  0 non-null      float64
 4   Unnamed: 4  0 non-null      float64
 5   행복          1 non-null      float64
 6   중립          0 non-null      float64
 7   슬픔          0 non-null      float64
 8   공포          0 non-null      float64
 9   혐오          0 non-null      float64
 10  분노          0 non-null      float64
 11  놀람          0 non-null      float64
dtypes: float64(9), object(3)
memory usage: 5.1+ MB


In [ ]:
chatbot_data_2.isnull().sum() # 결측치 확인

Unnamed: 0    45628
Unnamed: 1        1
Unnamed: 2        7
Unnamed: 3    55629
Unnamed: 4    55629
행복            55628
중립            55629
슬픔            55629
공포            55629
혐오            55629
분노            55629
놀람            55629
dtype: int64

In [ ]:
chatbot_data_2 = chatbot_data_2.iloc[1:, :3] # 필요없는 열 삭제
chatbot_data_2

,Unnamed: 0,Unnamed: 1,Unnamed: 2
1,S,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노
2,NaN,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,혐오
3,NaN,손님 왔어요.,중립
4,NaN,손님? 누구?,중립
5,NaN,몰라요. 팀장님 친구래요.,중립
...,...,...,...
55624,S,얘긴 다 끝났냐? 원예부,중립
55625,NaN,"예. 그거 때문에, 부탁이 있......는......데요.",중립
55626,NaN,여자 숨겨달라는거면 사절이다.,중립
55627,NaN,아무래도 안되나요?,중립


In [ ]:
chatbot_data_2[chatbot_data_2['Unnamed: 0'] =='S']

,Unnamed: 0,Unnamed: 1,Unnamed: 2
1,S,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노
8,S,그래서... 무슨 일 해?,중립
16,S,언제부터 시작할 수 있어?,중립
30,S,안 내켜.,혐오
42,S,"집 짓는 이유가 그렇게 중요해? 왜, 내가 사기라도 칠까봐?",혐오
...,...,...,...
55604,S,자네는 대체 뭘 하러 왔나! 젖은 생쥐 꼴이 된 나를 보면서 비웃으러 왔나?,분노
55609,S,조사결과에 따르면 의뢰인의 아버지는 횡령같은 걸 하지 않았다고 하는군.,중립
55614,S,그게 무슨 소리야?,중립
55619,S,기억해 두게. 이 사건에서 사실은 필요치 않아.,중립


In [ ]:
chatbot_data_2 = chatbot_data_2.iloc[:,1:] # Unnamed: 0 열 삭제
chatbot_data_2

,Unnamed: 1,Unnamed: 2
1,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노
2,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,혐오
3,손님 왔어요.,중립
4,손님? 누구?,중립
5,몰라요. 팀장님 친구래요.,중립
...,...,...
55624,얘긴 다 끝났냐? 원예부,중립
55625,"예. 그거 때문에, 부탁이 있......는......데요.",중립
55626,여자 숨겨달라는거면 사절이다.,중립
55627,아무래도 안되나요?,중립


In [ ]:
chatbot_data_2['Unnamed: 2'].value_counts()

중립     43786
놀람      4866
분노      3628
슬픔      1972
행복      1030
혐오       220
공포        98
ㅍ         12
분          4
ㅈ중립        1
분ㄴ         1
중림         1
ㄴ중립        1
줄          1
Name: Unnamed: 2, dtype: int64

In [ ]:
chatbot_data_2[chatbot_data_2['Unnamed: 2'] == 'ㅍ'] # 확인

,Unnamed: 1,Unnamed: 2
25037,인터넷에다 팔아야 되나.. 근데 너 이제 맘 잡았나보다?,ㅍ
27177,일부러 그럴 리가 있어요?,ㅍ
27948,그럼요?,ㅍ
28090,샤프심이 없어서..가지러 왔다가 누나 보이길래..,ㅍ
28455,아~ 자기소개서 쓰기 힘들어 죽겠어요~,ㅍ
28482,원래 남자들..다른 남자랑 친하게 지내면 그러니까..,ㅍ
36684,"그럼, 임마! 내가 누구냐? 한번 실수는 병가지상사다~이러시면서 나의 인간성과 품위...",ㅍ
37261,"처리 할거야. 처리해야지, 그게 돈이 얼만데?",ㅍ
37428,"와, 어디서 꽁돈이라도 생겼나?",ㅍ
37864,"엄마, 엄만 결혼이 뭐 그리 좋아?",ㅍ


In [ ]:
# 90 이하인 카테고리 추출
value_counts = chatbot_data_2['Unnamed: 2'].value_counts()
categories_to_remove = value_counts[value_counts <= 90].index

# 해당 카테고리를 포함하지 않는 행만 선택
chatbot_data_2 = chatbot_data_2[~chatbot_data_2['Unnamed: 2'].isin(categories_to_remove)]

In [ ]:
chatbot_data_2['Unnamed: 2'].value_counts()

중립    43786
놀람     4866
분노     3628
슬픔     1972
행복     1030
혐오      220
공포       98
Name: Unnamed: 2, dtype: int64

In [ ]:
chatbot_data_2.columns =['Sentence', 'Emotion']

In [ ]:
chatbot_data_2

,Sentence,Emotion
1,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노
2,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,혐오
3,손님 왔어요.,중립
4,손님? 누구?,중립
5,몰라요. 팀장님 친구래요.,중립
...,...,...
55624,얘긴 다 끝났냐? 원예부,중립
55625,"예. 그거 때문에, 부탁이 있......는......데요.",중립
55626,여자 숨겨달라는거면 사절이다.,중립
55627,아무래도 안되나요?,중립


In [ ]:
chatbot_data_1['Emotion'].unique()

array(['공포', '놀람', '분노', '슬픔', '중립', '행복', '혐오'], dtype=object)

In [ ]:
chatbot_data_2['Emotion'].unique()
chatbot_data_2 = chatbot_data_2.dropna(how='any')# Null 값 제거

In [ ]:
chatbot_data_2['Emotion'].value_counts()

중립    43786
놀람     4866
분노     3628
슬픔     1972
행복     1030
혐오      220
공포       98
Name: Emotion, dtype: int64

In [ ]:
chatbot_data_2[chatbot_data_2['Emotion'] == '중립'] # 확인

,Sentence,Emotion
3,손님 왔어요.,중립
4,손님? 누구?,중립
5,몰라요. 팀장님 친구래요.,중립
6,내 친구? 친구 누구?,중립
8,그래서... 무슨 일 해?,중립
...,...,...
55624,얘긴 다 끝났냐? 원예부,중립
55625,"예. 그거 때문에, 부탁이 있......는......데요.",중립
55626,여자 숨겨달라는거면 사절이다.,중립
55627,아무래도 안되나요?,중립


In [ ]:
len(chatbot_data_1), len(chatbot_data_2)

(38594, 55600)

In [ ]:
df_data = pd.concat([chatbot_data_1,chatbot_data_2], ignore_index=True)
df_data # 최종 데이터

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포
...,...,...
94189,얘긴 다 끝났냐? 원예부,중립
94190,"예. 그거 때문에, 부탁이 있......는......데요.",중립
94191,여자 숨겨달라는거면 사절이다.,중립
94192,아무래도 안되나요?,중립


In [ ]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94194 entries, 0 to 94193
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  94194 non-null  object
 1   Emotion   94194 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


# 데이터셋 전처리

In [ ]:
target_classes ={
    '공포': 0,
    '놀람': 1,
    '분노': 2,
    '슬픔': 3,
    '중립': 4,
    '행복': 5,
    '혐오': 6
  }

In [ ]:
# 레이블링
for cls in target_classes:
  df_data.loc[(df_data['Emotion'] == cls), 'Emotion'] = target_classes[cls]

In [ ]:
data_list =[]
# ['sentence', 'class'] 형태로 변환
for q, label in zip(df_data['Sentence'], df_data['Emotion']):
  data =[]
  data.append(q)
  data.append(str(label))

  data_list.append(data)

In [ ]:
print(data_list[6000])

['기술적으로도 아직도 해체해서 다시 완벽히 돌려놓는게 어려운데 해체를한다고?', '1']


In [ ]:
# BERTSentenceTransform 수정
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length, vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a) # 수정한 부분
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# 하이퍼 파라미터 설정
max_len = 100
batch_size = 64
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
random_seed = 42

# wandb 설정

In [ ]:
# wandb 설치
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.8/218.8 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=29743a94cfbeab9737249332f06f6ebdfae69e8834ef54594b714723280b3789
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
!wandb login 09b76e2d7910cb85b58be240c8cf088ff601d126

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import wandb
# project: 폴더 이름, entity: team_5g (팀), name: 기록명
wandb.init(project="koBERT",name=f"epoch_{num_epochs}_batch_size_{batch_size}_learning_rate_{learning_rate}",notes=f"koBERT-epoch_{num_epochs}-batch_size_{batch_size}-random_seed{random_seed}-learning_rate_{learning_rate}", entity='team_5g')

In [ ]:
# config
wandb.config ={
  "learning_rate": learning_rate,
  "epochs": num_epochs,
  "batch_size": batch_size,
  "seed": random_seed
}

In [ ]:
# loss 추적
wandb.define_metric('train_loss', summary='min')
wandb.define_metric('val_loss', summary='min')
# accuracy score 추적
wandb.define_metric('train_accuracy', summary='max')
wandb.define_metric('val_accuracy', summary='max')

In [ ]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
# torch GPU 설정
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_type)
device

device(type='cuda')

In [ ]:
# kobert 공식 git에 있는 get_kobert_model 선언
def get_kobert_model(model_path, vocab_file, ctx=device):
    bertmodel = BertModel.from_pretrained(model_path)
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,
                                                         padding_token='[PAD]')
    return bertmodel, vocab_b_obj

In [ ]:
import gluonnlp as nlp
from transformers import BertModel
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

In [ ]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = random_seed)

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 2)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = len(target_classes),   # 클래스 현재 7개
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate = 0.5).to(device)

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 모델 학습

In [ ]:
import time

# train_history = []
# test_history = []
# loss_history = []
y_preds =[]
results ={
      "train_loss": [],
      "train_acc": [],
      "val_loss": [],
      "val_acc": []
  }

# watch model
wandb.watch(model)
for e in range(num_epochs):
  train_acc = 0.0
  test_acc = 0.0

  model.train() #train
  start_time = time.time() # time

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
    optimizer.zero_grad()
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)

    y_pred = model(token_ids, valid_length, segment_ids)

    # print(label.shape, out.shape)
    train_loss = loss_fn(y_pred, label)
    train_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()
    scheduler.step()  # Update learning rate schedule
    train_acc += calc_accuracy(y_pred, label)

    if batch_id % log_interval == 0:
      print("epoch {} batch id {} loss {:.4f} train acc {:.4f}".format(e+1, batch_id+1, train_loss.data.cpu().numpy(), train_acc / (batch_id+1)))
      # train_history.append(train_acc / (batch_id+1))
      # loss_history.append(loss.data.cpu().numpy())
      results["train_loss"].append(train_loss.data.cpu().numpy())
      results["train_acc"].append(train_acc / (batch_id+1))
    # wandb log
    wandb.log({"train_acc": train_acc / (batch_id+1)}, commit=False)
    wandb.log({"train_loss": train_loss.data.cpu().numpy()}, commit=False)

  print("epoch {} train acc {:.4f}".format(e+1, train_acc / (batch_id+1)))

  model.eval() # inference
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    test_pred = model(token_ids, valid_length, segment_ids)
    y_preds.append(test_pred.cpu()) # for confusion matrix

    test_loss = loss_fn(test_pred, label)
    test_loss.backward()

    test_acc += calc_accuracy(test_pred, label)
    # wandb log
    wandb.log({"val_acc": test_acc/(batch_id+1)}, commit=False)
    wandb.log({"val_loss": test_loss.data.cpu().numpy()})

    results["val_loss"].append(test_loss.data.cpu().numpy())
    results["val_acc"].append(test_acc/(batch_id+1))

  end_time = time.time()
  print("epoch {} test acc {:.4f}".format(e+1, test_acc / (batch_id+1)))
  print("Time: {:.4f}sec".format((end_time - start_time)))

  # wandb log
  # wandb.log({'train_loss': results["train_loss"], 'val_loss': results["val_loss"], 'train_accuracy': results["train_acc"], 'val_accuracy': results["val_acc"]}, step=e)
  # test_history.append(test_acc / (batch_id+1))

y_pred_tensor = torch.cat(y_preds)

  0%|          | 0/1178 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9255 train acc 0.1719
epoch 1 batch id 201 loss 1.2361 train acc 0.4617
epoch 1 batch id 401 loss 1.2865 train acc 0.5305
epoch 1 batch id 601 loss 0.8878 train acc 0.5661
epoch 1 batch id 801 loss 1.0059 train acc 0.5873
epoch 1 batch id 1001 loss 1.1066 train acc 0.6029
epoch 1 train acc 0.6123


  0%|          | 0/295 [00:00<?, ?it/s]

epoch 1 test acc 0.6756
Time: 1429.0998sec


  0%|          | 0/1178 [00:00<?, ?it/s]

...

epoch 2 test acc 0.6901
Time: 1424.4253sec


  0%|          | 0/1178 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.6829 train acc 0.7344
epoch 3 batch id 201 loss 0.7235 train acc 0.7390
epoch 3 batch id 401 loss 0.9681 train acc 0.7498
epoch 3 batch id 601 loss 0.5405 train acc 0.7609
epoch 3 batch id 801 loss 0.5763 train acc 0.7673
epoch 3 batch id 1001 loss 0.8072 train acc 0.7718
epoch 3 train acc 0.7721


  0%|          | 0/295 [00:00<?, ?it/s]

epoch 3 test acc 0.6913
Time: 1423.4076sec


In [ ]:
torch.save(model.state_dict(), "./base_model_koBERT_epoch3_.pt")

# 키워드 추출
- 명사, 동사, 형용사

In [ ]:
!pip install konlpy

from konlpy.tag import Okt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 44.1 MB/s eta 0:00:00


In [ ]:
okt = Okt()
kor_sentence = '너무 배고파 ㅜㅜ'
print('OKT 품사 태깅 :', okt.pos(kor_sentence))

OKT 품사 태깅 : [('너무', 'Adverb'), ('배고파', 'Adjective'), ('ㅜㅜ', 'KoreanParticle')]


In [ ]:
# 명사, 동사, 형용사 추출하기
keyword = [x for (x, y) in okt.pos(kor_sentence) if y in ['Verb', 'Noun', 'Adjective']]
print('\nOKT 명사, 동사, 형용사 추출 :', keyword)


OKT 명사, 동사, 형용사 추출 : ['배고파']


In [ ]:
# okt.normalize(kor_sentence)

# 새로운 문장 테스트

In [ ]:
# target_classes ={
#     '공포': 0,
#     '놀람': 1,
#     '분노': 2,
#     '슬픔': 3,
#     '중립': 4,
#     '행복': 5,
#     '혐오': 6
#   }

In [ ]:
reverse =dict(map(reversed, target_classes.items()))
print(reverse)

{0: '공포', 1: '놀람', 2: '분노', 3: '슬픔', 4: '중립', 5: '행복', 6: '혐오'}


In [ ]:
list(reverse.keys())

[0, 1, 2, 3, 4, 5, 6]

In [ ]:
# 토큰화
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

def predict_and_keword(predict_sentence):
  new_data =[[predict_sentence, '0']]
  # print(new_data)

  new_dataset = BERTDataset(new_data, 0, 1, tokenizer, vocab ,max_len, True, False)
  new_dataloader = torch.utils.data.DataLoader(new_dataset, batch_size = batch_size, num_workers =5)

  model.eval()
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(new_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)

    test_eval =[]
    for logit in out:
      test_eval.append(reverse[np.argmax(logit.detach().cpu().numpy())])

    # 키워드 추출: 명사, 동사, 형용사 추출하기
    keyword = [x for (x, y) in okt.pos(predict_sentence) if y in ['Verb', 'Noun', 'Adjective']]
    print('\nOKT 명사, 동사, 형용사 추출 :', keyword)
    test_eval += keyword

    return test_eval

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
predict_and_keword('어제 헤어졌어 ㅜ')

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


  0%|          | 0/1 [00:00<?, ?it/s]


OKT 명사, 동사, 형용사 추출 : ['어제', '헤어졌어']


['슬픔', '어제', '헤어졌어']

In [ ]:
predict_and_keword('진짜 짜증나')

  0%|          | 0/1 [00:00<?, ?it/s]


OKT 명사, 동사, 형용사 추출 : ['진짜', '짜증나']


['분노', '진짜', '짜증나']

In [ ]:
predict_and_keword('너무 무서워')

  0%|          | 0/1 [00:00<?, ?it/s]


OKT 명사, 동사, 형용사 추출 : ['무서워']


['공포', '무서워']

# 성능 올리기
## 1. 파라메터 수정해서 테스트
## 2. 중립 데이터 제거해서 테스트
## 3. 모델 구조 수정?